# C/C++ __stdcall DLL, Excel VBA

* Change the current working directory into `./demo`

In [1]:
%cd demo

F:\SEU\SEE\PySEE\home\notebook\demo


In [2]:
%pwd

'F:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

## 1 C/C++ __stdcall DLL 

  For **Visual Basic applications** (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call **`__stdcall`** functions, not `__cdecl` functions. 
  
*  **`__stdcall`** creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from **right to left(从右向左)**

*  **`__declspec(dllexport)`** is used on an `exported` function in a DLL

For Example:

```c
__declspec(dllexport) double __stdcall  mean(double data[], int size);;
```



Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

**Example:** Equations for Region4 of [IAPWS-IF97](http://www.iapws.org/relguide/IF97-Rev.pdf)

**8 Equations for Region 4** 

* 8.1 The Saturation-Pressure Equation (Basic Equation) P33,Eq30
  
    
     pSat(double T);
  
* 8.2 The Saturation-Temperature Equation (Backward Equation) P35, Eq31
   
  
     TSat(double p);

```

### 1.1 Create DLL with `__stdcall` calling convention


**The ·MANY· headers**

When you create header files for your `__stdcall` DLLs,

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols


1.The headers for building the Windows **`__stdcall`** shared library  

```c
__declspec(dllexport) double __stdcall pSat(double T);
__declspec(dllexport) double __stdcall TSat(double p);
```
2.The header for building an executable that uses the  shared library.

```c
__declspec(dllimport) double __stdcall pSat(double T);
__declspec(dllimport) double __stdcall TSat(double p);
```


**The Multi-purpose header**

Create a DLL of IAPWS-IF97-Region4 with the following code:

* region4.h

* region4.c

The following header `region4.h`  declares the interface for 

* building the **Windows/Linux** shared library  

* building an executable that uses the  shared library.

Using the macro **PORT** as the return type of function 
 
 
* For Windows,**export** all functions as `__stdcall`.

```c
#define PORT __declspec(dllexport) double __stdcall
```

* For Windows,**import** all functions as `__stdcall`.

```c
#define PORT __declspec(dllimport) double __stdcall
```

* For Linux as  `__cdecl`

```c
#define PORT double
```

**#pragma once** 

This is a `non-standard` but widely supported preprocessor directive designed to cause the current source file to be included **only once** in a single compilation. 

**extern "C"** 

extern "C" syntax to allow the run-time library functions to be used in C++ programs.*

In [9]:
%%file ./include/region4.h
#pragma once

#ifdef __cplusplus
extern "C" {
#endif

#ifdef BUILD_DLL

    #ifdef WIN32
        #define PORT __declspec(dllexport) double __stdcall 
    #else
        #define PORT double
    #endif    

#else

    #ifdef WIN32
        #define PORT __declspec(dllimport) double __stdcall   
    #else
        #define PORT double
    #endif    

#endif

PORT pSat(double T);
PORT TSat(double p);
        
#ifdef __cplusplus
	}
#endif        

Overwriting ./include/region4.h


In [10]:
%%file ./src/region4.c
#include <math.h>
#include "region4.h"

//
//  Initialize coefficients for region 4
//
static double n[11] = {0, 0.11670521452767E+04, -0.72421316703206E+06, -0.17073846940092E+02,
                          0.12020824702470E+05, -0.32325550322333E+07, 0.14915108613530E+02,
                         -0.48232657361591E+04, 0.40511340542057E+06, -0.23855557567849E+00,
                          0.65017534844798E+03};

PORT pSat(double T)
// saturation pressure of water
// pSat in MPa
// T :temperaturein K
//
// pSat = -1: temperature outside range
//
{
    double pS;
    if (T < 273.15 || T > 647.096) // tc_water=647.096
        pS = -1.0;
    else
    {
        double del = T + n[9] / (T - n[10]);
        double aco = del * (del + n[1]) + n[2];
        double bco = del * (n[3] * del + n[4]) + n[5];
        double cco = del * (n[6] * del + n[7]) + n[8];
        pS = pow(2 * cco / (-bco + sqrt(bco * bco - 4 * aco * cco)), 4);
    }
    return pS;
}

PORT TSat(double p)
// saturation temperature of water
// TSat in K
// p :pressure MPa
//
// TSat=-1: pressure outside range
//
{
    double TS;
    if (p < 0.000611212677 || p > 22.064)
        TS = -1.0;
    else
    {
        double bet = pow(p, 0.25);
        double eco = bet * (bet + n[3]) + n[6];
        double fco = bet * (n[1] * bet + n[4]) + n[7];
        double gco = bet * (n[2] * bet + n[5]) + n[8];
        double dco = 2.0 * gco / (-fco - sqrt(fco * fco - 4.0 * eco * gco));
        TS = 0.5 * (n[10] + dco - sqrt((n[10] + dco) * (n[10] + dco) - 4.0 * (n[9] + n[10] * dco)));
    }
    return TS;
}

Overwriting ./src/region4.c


In [11]:
!gcc -c -DBUILD_DLL -o ./obj/region4.o ./src/region4.c -I./include
!gcc -shared -o ./bin/libregion4.dll ./obj/region4.o  -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an **undecorated** alias for the `exported function names` that is simply **the name of the function** 

### 1.2  Add -static-libgcc ,-output-def=libregion4.def

 Links the GNU `libgcc` library `statically`

In [12]:
!gcc -c -DBUILD_DLL -o ./obj/region4.o ./src/region4.c -I./include
!gcc -shared -o ./bin/libregion4.dll -static-libgcc ./obj/region4.o -Wl,--add-stdcall-alias,-output-def=./bin/libregion4.def

**-static-libgcc**

* This option links the GNU `libgcc` library **statically** 
   
**-output-def=libregion4.def**

* Name of `.def` file to be created.
    
* **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./bin/libregion4.def

In [14]:
%%file ./makefile-region4.mk

CC=gcc
CFLAGS=-DBUILD_DLL

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: libregion4

libregion4: obj
	 $(CC) -shared -o $(BINDIR)libregion4.dll -static-libgcc $(OBJDIR)region4.o -Wl,--add-stdcall-alias,-output-def=libregion4.def
obj: 
	 $(CC) -c $(CFLAGS) -o $(OBJDIR)region4.o $(SRCDIR)region4.c -I$(INCDIR)


Overwriting ./makefile-region4.mk


In [15]:
!make -f makefile-region4.mk

gcc -shared -o ./bin/libregion4.dll -static-libgcc ./obj/region4.o -Wl,--add-stdcall-alias,-output-def=libregion4.def


### 1.3 Call Dll from C/C++

In [6]:
%%file ./src/mainReg4.c

#include <stdio.h> 
#include "region4.h"

int main() {
     double T=300.0;
     printf("Saturation P is %f\n", pSat(T));  
     return 0;
}

Overwriting ./src/mainReg4.c


In [16]:
!gcc -c -o ./obj/mainReg4.o ./src/mainReg4.c -I./include/
!gcc -o  ./bin/mainReg4 ./obj/mainReg4.o  -L./bin/ -lregion4

In [17]:
!.\bin\mainReg4

Saturation P is 0.003537


### 1.4 Python API

* `__stdcall` calling convention: **windll, WINFUNCTYPE**

In [18]:
%%file ./src/region4.py

from ctypes import windll,c_double,WINFUNCTYPE

flib = windll.LoadLibrary('./bin/libregion4.dll')
prototype = WINFUNCTYPE(c_double,c_double)

def pSat(T):
    f = prototype(("pSat", flib),)
    return f(T)

def TSat(p):
    f = prototype(("TSat", flib),)
    return f(p)

Overwriting ./src/region4.py


**add `mathfuns.py` into the interperte search path**

In [19]:
import sys
sys.path.append('./src')

In [20]:
from region4 import *
T=300
p=pSat(T)
print(p)

0.003536589413013015


In [21]:
%%file ./src/test4.py

import unittest
from region4 import *

class Region4Test (unittest.TestCase):

    def setUp(self):
        # IF97-dev,Table35 Page 34 : T(K) p(MPa)
        self.tab35=[[300, 0.353658941e-2],
                    [500, 0.263889776e1],
                    [600, 0.123443146e2]]

        # IF97-dev, Table 36 Page 36 :  p(MPa) T(K)
        self.tab36=[[0.1, 0.372755919e3],
                    [  1, 0.453035632e3],
                    [ 10, 0.584149488e3]]

    def test_pSat(self):
        places = 6
        for item in  self.tab35:
             self.assertAlmostEqual(pSat(item[0]),item[1],places)

    def test_TSat(self):
        places = 6
        for item in  self.tab36:
             self.assertAlmostEqual(TSat(item[0]),item[1],places)

if __name__ == '__main__':
    unittest.main()       

Overwriting ./src/test4.py


In [22]:
%run ./src/test4.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


## 2 Call DLL from Excel VBA(64bits)

Do as the following steps:

### 2.1 Put  DLL in the default path of Windows'dll 


`libregion4.dll`  in `C:\windows\system`


### 2.2 Create the xlsm

`demo-r4.xlsm` in `./demo/ExcelVBA/`


### 2.3 VBA module to call the DLL.

There are a few ways to open up the **VBA Editor** in Excel.

* From the `Developer Tools` tab, you can click the `Visual Basic` button.

* A keyboard shortcut: press **"Alt+F11"** :

![vba](./img/vba.jpg)

Into **VBA** Editor, then， create the module **mathfuns** to call library:

```VBA
Declare PtrSafe Function pSat Lib "libregion4" (ByVal T As Double) As Double
Declare PtrSafe Function TSat Lib "libregion4" (ByVal p As Double) As Double

Public Function CalpSat(ByVal T As Double) As Double
    CalpSat = pSat(T)
End Function
 
Public Function  CalTSat(ByVal p As Double) As Double
    CalTSat = TSat(p)
End Function
```

### 2.4  call VBA methods in cells

![demo-r4](./img/demo-r4.jpg)

## 3 Excel4Engineering

Examples of Excel VBA for Engineering: Analysizing the Ideal Rankine Cycle, Monitoring Industrial Process 

https://github.com/thermalogic/Excel4Engineering
    

## Reference

**Windows's DLL**

* [What is a DLL and How Do I Create or Use One?](http://www.mingw.org/wiki/DLL)

* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

    
**Microsoft Excel**

* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html

* [Language reference for Visual Basic for Applications(VBA)](https://docs.microsoft.com/en-us/office/vba/api/overview/language-reference)

* [Excel VBA reference](https://docs.microsoft.com/en-us/office/vba/api/overview/excel)

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


